<a href="https://colab.research.google.com/github/hqu/DPI-852M/blob/main/Open_Weather_Map_demo_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from google.colab import userdata
api_key = userdata.get('openweathermap')

In [29]:
# prompt: get tomorrow's forecast for Boston in F using openweather map and parse response.text json into human readable forecast

import requests
import json
from datetime import datetime, timedelta

# Assuming api_key is defined and holds your OpenWeatherMap API key
# If not, replace with your actual API key
# api_key = "YOUR_API_KEY"  # Replace with your actual API key

city = "Boston"
url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=imperial"

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

    data = response.json()

    # Find tomorrow's forecast
    tomorrow = datetime.now() + timedelta(days=1)
    tomorrow_date_str = tomorrow.strftime("%Y-%m-%d")

    tomorrow_forecast = None
    for forecast in data["list"]:
        forecast_date_str = forecast["dt_txt"][:10]
        if forecast_date_str == tomorrow_date_str:
            tomorrow_forecast = forecast
            break

    if tomorrow_forecast:
        temp = tomorrow_forecast["main"]["temp"]
        feels_like = tomorrow_forecast["main"]["feels_like"]
        description = tomorrow_forecast["weather"][0]["description"]

        print(f"Tomorrow's forecast for {city}:")
        print(f"Temperature: {temp}°F")
        print(f"Feels like: {feels_like}°F")
        print(f"Conditions: {description}")

    else:
        print(f"No forecast found for tomorrow in {city}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching weather data: {e}")
except KeyError as e:
    print(f"Error parsing weather data: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Tomorrow's forecast for Boston:
Temperature: 22.91°F
Feels like: 16.25°F
Conditions: broken clouds


In [30]:
data

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1737720000,
   'main': {'temp': 17.06,
    'feels_like': 8.38,
    'temp_min': 17.06,
    'temp_max': 18.19,
    'pressure': 1017,
    'sea_level': 1017,
    'grnd_level': 1013,
    'humidity': 75,
    'temp_kf': -0.63},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02n'}],
   'clouds': {'all': 11},
   'wind': {'speed': 6.06, 'deg': 263, 'gust': 17.92},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'n'},
   'dt_txt': '2025-01-24 12:00:00'},
  {'dt': 1737730800,
   'main': {'temp': 22.75,
    'feels_like': 13.32,
    'temp_min': 22.75,
    'temp_max': 25.88,
    'pressure': 1018,
    'sea_level': 1018,
    'grnd_level': 1014,
    'humidity': 72,
    'temp_kf': -1.74},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01d'}],
   'clouds': {'all': 7},
   'wind': {'speed': 8.37, 'deg': 279, 'gust': 16.71},
   'visibility': 100

In [31]:
# prompt: save this weather into a new Google Sheet

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Create a new spreadsheet
sh = gc.create('Weather Data')

# Open the spreadsheet
worksheet = sh.sheet1

# Append the weather data to the spreadsheet
if tomorrow_forecast:
    worksheet.append_row([
        city,
        tomorrow_date_str,
        temp,
        feels_like,
        description
    ])
    print(f"Weather data appended to Google Sheet: {sh.url}")
else:
    print("No weather data to append.")

Weather data appended to Google Sheet: https://docs.google.com/spreadsheets/d/1gdV7W2uqJSoHLiQMlzsE-yIcFqovdlm_9fqRXUK-X1Y


In [32]:
# prompt: get the next 5 days forecast and save it to a new tab in the same google sheet. be sure to include the heading row with these variables: Date, Temp low, temp high, Summary, Precipitation probability, Precipitation type, Precipitation accumulation, Icon

from google.colab import userdata
import requests
import json
from datetime import datetime, timedelta
from google.colab import auth
import gspread
from google.auth import default

api_key = userdata.get('openweathermap')
city = "Boston"
url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=imperial"

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    sh = gc.open('Weather Data')
    worksheet_new = sh.add_worksheet(title="5-Day Forecast", rows="100", cols="20")
    worksheet_new.append_row(["Date", "Temp low", "Temp high", "Summary", "Precipitation probability", "Precipitation type", "Precipitation accumulation", "Icon"])

    for i in range(0, 5):
        forecast = data["list"][i*8] #access 3 hour intervals
        date_str = forecast["dt_txt"][:10]
        temp_min = forecast["main"]["temp_min"]
        temp_max = forecast["main"]["temp_max"]
        summary = forecast["weather"][0]["description"]
        try:
          pop = forecast["pop"]
        except:
          pop = 0
        try:
          precipitation_type = forecast["weather"][0]["main"]
        except:
          precipitation_type = "N/A"
        try:
            precipitation_accumulation = forecast["rain"]["3h"]
        except KeyError:
            try:
              precipitation_accumulation = forecast["snow"]["3h"]
            except KeyError:
              precipitation_accumulation = 0
        icon = forecast["weather"][0]["icon"]
        worksheet_new.append_row([date_str, temp_min, temp_max, summary, pop, precipitation_type, precipitation_accumulation, icon])

except requests.exceptions.RequestException as e:
    print(f"Error fetching weather data: {e}")
except KeyError as e:
    print(f"Error parsing weather data: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [33]:
# prompt: add a new column into the 5-Day Forecast Google Sheet, by combining the variable Icon with in between these 2 strings: "https://openweathermap.org/img/wn/" + icon + ".png"

import gspread
from google.auth import default
from google.colab import auth

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
sh = gc.open('Weather Data')
worksheet_new = sh.worksheet("5-Day Forecast")

# Get all values from the worksheet
all_values = worksheet_new.get_all_values()

# Find the index of the "Icon" column
header_row = all_values[0]
icon_index = header_row.index("Icon")

# Insert a new column named "Icon URL" next to the "Icon" column
header_row.insert(icon_index + 1, "Image URL")
worksheet_new.update("A1", [header_row])  # Update the header row


# Iterate through the rows and create the Icon URLs
for i in range(1, len(all_values)):
    icon = all_values[i][icon_index]
    icon_url = f"https://openweathermap.org/img/wn/{icon}.png"
    all_values[i].insert(icon_index+1, icon_url)

# Update the worksheet with the new column
worksheet_new.update("A1", all_values)

<ipython-input-33-cf82cd0b7a8e>:22: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet_new.update("A1", [header_row])  # Update the header row
<ipython-input-33-cf82cd0b7a8e>:32: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet_new.update("A1", all_values)


{'spreadsheetId': '1gdV7W2uqJSoHLiQMlzsE-yIcFqovdlm_9fqRXUK-X1Y',
 'updatedRange': "'5-Day Forecast'!A1:I6",
 'updatedRows': 6,
 'updatedColumns': 9,
 'updatedCells': 54}

In [34]:
# prompt: Use openweathermap API to get only tomorrow's forecast for Boston in F

import requests
import json
from datetime import datetime, timedelta

api_key = "YOUR_API_KEY"  # Replace with your actual API key
city = "Boston"
url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=imperial"

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    tomorrow = datetime.now() + timedelta(days=1)
    tomorrow_date_str = tomorrow.strftime("%Y-%m-%d")

    tomorrow_forecast = None
    for forecast in data["list"]:
        forecast_date_str = forecast["dt_txt"][:10]
        if forecast_date_str == tomorrow_date_str:
            tomorrow_forecast = forecast
            break

    if tomorrow_forecast:
        temp = tomorrow_forecast["main"]["temp"]
        feels_like = tomorrow_forecast["main"]["feels_like"]
        description = tomorrow_forecast["weather"][0]["description"]

        print(f"Tomorrow's forecast for {city}:")
        print(f"Temperature: {temp}°F")
        print(f"Feels like: {feels_like}°F")
        print(f"Conditions: {description}")
    else:
        print(f"No forecast found for tomorrow in {city}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching weather data: {e}")
except KeyError as e:
    print(f"Error parsing weather data: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error fetching weather data: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/data/2.5/forecast?q=Boston&appid=YOUR_API_KEY&units=imperial


# END
## Go see Open Weather Map demo part 2 - updating the data
https://colab.research.google.com/drive/1HJYG7ORMuQtRYVcvoudPklwsKbQGKJeX#scrollTo=PsyfINGIcyrs